# Price Prediction
- __The Goal__: We have data scraping from Craigslist every hour gathering data on apartments in New York City. I would like to take that data and build a model that predicts the price of an apartment. This can be used similar to a Zestimate on Zillow for being a reference for if an apartment is a good "deal".


- __What This Notebook Is__: We've already conducted an exploratory analysis of the data. Now we will use what we learned to build a predictive model. The ultimate goal is to get this into a functioning web app, so many of the functions used in this exploratory model building will need to be exported in order to handle incoming data. This notebook will build our model that we can export via pickle, and also define many of these functions for encoding incoming data.

# Import Our Libraries

In [1]:
import pandas as pd
import numpy as np

# Scikit libraries


# Read in our data


In [2]:
df = pd.read_csv('../../notebooks/housing_cleaned.csv')

In [3]:
df.head()

,Unnamed: 0,id,address,area,bedrooms,bikeScore,datetime,distanceToNearestIntersection,has_image,has_map,...,includes_area,year,month,dow,day,hour,advertises_no_fee,is_repost,sideOfStreetEncoded,postalCodeChopped
0,0,6911917730,320 Chauncey St,NaN,3.0,64.0,2019-06-21 14:34:00,0.000000,1,1,...,0,2019,6,4,21,14,1,0,1.0,11233.0
1,1,6917210186,530 W 143rd St,800.0,1.0,88.0,2019-06-21 14:33:00,203.483553,1,1,...,1,2019,6,4,21,14,0,1,0.0,10031.0
2,2,6914527887,410 Pulaski St,NaN,3.0,79.0,2019-06-21 14:33:00,0.013114,1,1,...,0,2019,6,4,21,14,0,0,1.0,11221.0
3,3,6914529944,410 Pulaski St,NaN,3.0,79.0,2019-06-21 14:33:00,0.013114,1,1,...,0,2019,6,4,21,14,1,0,1.0,11221.0
4,4,6917173545,4754 Center Blvd,653.0,1.0,81.0,2019-06-21 14:33:00,61.301497,1,1,...,1,2019,6,4,21,14,1,1,0.0,11109.0
